In [5]:
import cv2
import numpy as np

def richardson_lucy(image, psf, iterations, clip=True):
    # Initialize the estimated image
    estimated_image = np.ones(image.shape)

    for i in range(iterations):
        # Calculate the error between the observed and estimated images
        observed = cv2.filter2D(estimated_image, -1, psf) + 1e-10
        error = image / observed
        
        # Update the estimated image using the error
        estimated_image *= cv2.filter2D(error, -1, cv2.flip(psf, -1))

        # Apply non-negativity constraint to estimated image
        estimated_image[estimated_image < 0] = 0
        
        # Normalize the estimated image to maintain consistent intensity
        estimated_image = estimated_image * (image.sum() / estimated_image.sum())

    if clip:
        # Clip to ensure pixel values are in the valid range [0, 255]
        estimated_image = np.clip(estimated_image, 0, 255).astype(np.uint8)

    return estimated_image

psf = np.array([[0.05, 0.2, 0.05],
                [0.2, 0.5, 0.2],
                [0.05, 0.2, 0.05]])

# Load the degraded image
degraded_image = cv2.imread('Lincoln.jpg', cv2.IMREAD_GRAYSCALE)

# Define the point spread function (PSF) - You need to provide this based on your specific case
# Example PSF: psf = np.array([[0.05, 0.2, 0.05], [0.2, 0.5, 0.2], [0.05, 0.2, 0.05]])

# Define the number of Richardson-Lucy iterations - Adjust this value based on your case
iterations = 10

# Restore the image using the Richardson-Lucy algorithm
restored_image = richardson_lucy(degraded_image, psf, iterations)

# Save or display the restored image
cv2.imwrite('restored_image.jpg', restored_image)
cv2.imshow('Restored Image', restored_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

[[238 240 235 ... 240 240 236]
 [237 240 236 ... 237 236 240]
 [238 239 239 ... 236 237 240]
 ...
 [235 241 236 ... 239 238 237]
 [239 237 239 ... 238 238 238]
 [239 237 239 ... 238 238 238]]


In [4]:
import cv2
import numpy as np

# Richardson-Lucy deconvolution function
def richardson_lucy_deconv(noisy_image, point_spread_function, number_of_iterations):

    # Initialize restored image
    restored_image = np.ones(noisy_image.shape)

    # Perform iterations
    for i in range(number_of_iterations):

        # Convolve estimated image with point spread function
        convolved = cv2.filter2D(restored_image, -1, point_spread_function)

        # Calculate error
        error = noisy_image / (convolved + 1e-6)

        # New estimate using point spread function transpose
        restored_image = restored_image * cv2.filter2D(error, -1, cv2.flip(point_spread_function, -1))

        # Clip values
        restored_image = np.clip(restored_image, 0, 255)

    # Return restored image
    return restored_image

# Load degraded image
degraded_image = cv2.imread('Lincoln.jpg', 0)

if degraded_image is not None:
    # Create synthetic point spread function
    point_spread_function = np.ones((5, 5), np.float32) / 25

    # Number of iterations
    number_of_iterations = 25

    # Restore image
    restored_image = richardson_lucy_deconv(degraded_image, point_spread_function, number_of_iterations)

    # Save restored image
    cv2.imwrite('restoredImage.png', restored_image)

    # Display restored image
    cv2.imshow('Restored Image', restored_image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
else:
    print("Error: Failed to load the degraded image.")
